# scala_spark 统计每日访问量

In [1]:
sc.master

local

In [15]:
val spark = SparkSession
      .builder()
      .master("local[2]")
      .appName("WindowTest")
      .getOrCreate()
//    import spark.implicits._

spark = org.apache.spark.sql.SparkSession@79cef4c2


org.apache.spark.sql.SparkSession@79cef4c2

创建模拟用户访问表

In [23]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._
// 聚合函数都来自于func
import org.apache.spark.sql.functions.row_number
import org.apache.spark.sql.functions.dense_rank
import org.apache.spark.sql.expressions.Window
// pyspark:pyspark.sql.window.Window

In [5]:
val UserAcessLog = Array("2019-05-29,aa,45","2019-05-29,bb,45",
                         "2019-05-29,aa,45","2019-05-29,cc,45",
                         "2019-05-30,aa,35","2019-05-30,ee,35",
                         "2019-05-30,aa,35","2019-05-30,aa,35"
                        )
val LogStrRDD = sc.parallelize(UserAcessLog,1)
                .filter(line => if(line.split(",")(2).length >0) true else false)
                .map(line=> 
                     Row(line.split(",")(0), 
                         line.split(",")(1),
                         line.split(",")(2).toInt))

val schema = StructType(Array(StructField("date",StringType),
                             StructField("userid",StringType),
                             StructField("click",IntegerType))
                       )


UserAcessLog = Array(2019-05-29,aa,45, 2019-05-29,bb,45, 2019-05-29,aa,45, 2019-05-29,cc,45, 2019-05-30,aa,35, 2019-05-30,ee,35, 2019-05-30,aa,35, 2019-05-30,aa,35)
LogStrRDD = MapPartitionsRDD[2] at map at <console>:43
schema = StructType(StructField(date,StringType,true), StructField(userid,StringType,true), StructField(click,IntegerType,true))


StructType(StructField(date,StringType,true), StructField(userid,StringType,true), StructField(click,IntegerType,true))

In [22]:
LogStrRDD.take(5)

Array([2019-05-29,aa,45], [2019-05-29,bb,45], [2019-05-29,aa,45], [2019-05-29,cc,45], [2019-05-30,aa,35])

In [6]:
val LogDF = spark.sqlContext.createDataFrame(LogStrRDD,schema)

LogDF = [date: string, userid: string ... 1 more field]


[date: string, userid: string ... 1 more field]

In [7]:
LogDF.show()

+----------+------+-----+
|      date|userid|click|
+----------+------+-----+
|2019-05-29|    aa|   45|
|2019-05-29|    bb|   45|
|2019-05-29|    aa|   45|
|2019-05-29|    cc|   45|
|2019-05-30|    aa|   35|
|2019-05-30|    ee|   35|
|2019-05-30|    aa|   35|
|2019-05-30|    aa|   35|
+----------+------+-----+



In [18]:
LogDF.groupBy("date").agg(
    countDistinct("userid").as("uv"),
    sum("click").alias("click_sum")).show()


+----------+---+---------+
|      date| uv|click_sum|
+----------+---+---------+
|2019-05-29|  3|      180|
|2019-05-30|  2|      140|
+----------+---+---------+



# 开窗函数：分组求topn

In [16]:
import spark.implicits._

In [35]:
// 这种写法在Pyspark可以，Python增加字段只能用withColumn
// LogDF.select("date","userid",LogDF("click")+10)


Name: Syntax Error.
Message: 
StackTrace: 

In [22]:
val rankSpec = Window.partitionBy("date").orderBy(LogDF("click").desc)


rankSpec = org.apache.spark.sql.expressions.WindowSpec@1e3d2eb7


lastException: Throwable = null


org.apache.spark.sql.expressions.WindowSpec@1e3d2eb7

In [28]:
LogDF.withColumn("rank", dense_rank().over(rankSpec)).show()


+----------+------+-----+----+
|      date|userid|click|rank|
+----------+------+-----+----+
|2019-05-29|    aa|   45|   1|
|2019-05-29|    bb|   45|   1|
|2019-05-29|    aa|   45|   1|
|2019-05-29|    cc|   45|   1|
|2019-05-30|    aa|   35|   1|
|2019-05-30|    ee|   35|   1|
|2019-05-30|    aa|   35|   1|
|2019-05-30|    aa|   35|   1|
+----------+------+-----+----+



In [40]:
// 这种写法在Pyspark可以，Python增加字段只能用withColumn
/*
*LogDF.select("date","userid","click",
*             dense_rank().over(
*                 Window.partitionBy("date")
*                     .orderBy("click")).alias("aa"))
*/

Name: Syntax Error.
Message: 
StackTrace: 